# Results p. 5
## Saccade Discriminability
### How sensitive are detectors to saccade onsets/offsets?

In [38]:
import os
import copy

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import peyes

from analysis._article_results.lund2013._helpers import *
import analysis.statistics.channel_sdt as ch_sdt

# pio.renderers.default = "browser"
FIG_ID, IS_SUPP = 5, False

## Load Data

In [24]:
LABEL = 2       # EventLabelEnum.SACCADE.value
THRESHOLD = 5   # samples

csdt_metrics = ch_sdt.load(
    dataset_name=DATASET_NAME,
    output_dir=PROCESSED_DATA_DIR,
    label=LABEL,
    stimulus_type=STIMULUS_TYPE,
    channel_type=None,
)
csdt_metrics.drop(index=['P', 'PP', 'N', 'TP'], level=peyes.constants.METRIC_STR, inplace=True)    # Remove unused metrics

csdt_metrics

trial_id                                       25                      \
gt                                             RA                       
pred                                           MN   engbert  remodnav   
channel_type metric           threshold                                 
onset        recall           0          0.760000  0.280000  0.120000   
             precision        0          0.730769  0.225806  0.230769   
             f1               0          0.745098  0.250000  0.157895   
             false_alarm_rate 0          0.001410  0.004836  0.002015   
             d_prime          0          3.692915  2.004514  1.700831   
...                                           ...       ...       ...   
offset       precision        20         0.961538  0.806452  1.000000   
             f1               20         0.980392  0.892857  0.684211   
             false_alarm_rate 20         0.010346  0.062074  0.000000   
             d_prime          20         4.495950  3.887074  2.454317   
             criterion        20        -0.157000 -0.461438  1.177817   

trial_id                                                               \
gt                                                                      
pred                                         idvt        nh       idt   
channel_type metric           threshold                                 
onset        recall           0          0.000000  0.000000  0.000000   
             precision        0          0.000000  0.000000  0.000000   
             f1               0               NaN       NaN       NaN   
             false_alarm_rate 0          0.003425  0.005239  0.003425   
             d_prime          0         -0.854360 -0.992838 -0.854360   
...                                           ...       ...       ...   
offset       precision        20         1.000000  0.846154  1.000000   
             f1               20         0.809524  0.862745  0.809524   
             false_alarm_rate 20         0.000000  0.041383  0.000000   
             d_prime          20         2.864548  2.909847  2.864548   
             criterion        20         0.972701  0.279937  0.972701   

trial_id                                                               \
gt                                                                 MN   
pred                                         ivvt       ivt        RA   
channel_type metric           threshold                                 
onset        recall           0          0.040000  0.040000  0.730769   
             precision        0          0.031250  0.031250  0.760000   
             f1               0          0.035088  0.035088  0.745098   
             false_alarm_rate 0          0.006246  0.006246  0.001209   
             d_prime          0          0.747234  0.747234  3.648512   
...                                           ...       ...       ...   
offset       precision        20         0.781250  0.781250  1.000000   
             f1               20         0.877193  0.877193  0.980392   
             false_alarm_rate 20         0.072420  0.072420  0.000000   
             d_prime          20         3.814531  3.814531  4.090681   
             criterion        20        -0.497709 -0.497709  0.359635   

trial_id                                           ...        44            \
gt                                                 ...        RA             
pred                                      engbert  ...      ivvt       ivt   
channel_type metric           threshold            ...                       
onset        recall           0          0.153846  ...  0.000000  0.000000   
             precision        0          0.129032  ...  0.000000  0.000000   
             f1               0          0.140351  ...       NaN       NaN   
             false_alarm_rate 0          0.005441  ...  0.015744  0.014220   
             d_prime          0          1.526366  ... -1.151515 -1.112344   
...       

## Onset Detection

In [25]:
onset_statistics, onset_pvalues, onset_nemenyi, onset_Ns = ch_sdt.friedman_nemenyi(
    csdt_metrics, "onset", THRESHOLD, [GT1, GT2]
)

onset_pvalues <= ALPHA

gt,MN,RA
metric,,
criterion,True,True
d_prime,True,True
f1,True,True
false_alarm_rate,True,True
precision,True,True
recall,True,True


In [26]:
pd.concat([onset_statistics, onset_pvalues], axis=1, keys=['Q', 'p']).stack(1, future_stack=True)

Q             p
metric           gt                         
criterion        MN  68.922280  6.800451e-13
                 RA  94.618314  3.319779e-18
d_prime          MN  66.093264  2.579425e-12
                 RA  85.240254  2.943650e-16
f1               MN  52.259067  1.653987e-09
                 RA  70.335449  3.489760e-13
false_alarm_rate MN  52.779944  1.299309e-09
                 RA  73.444874  8.019047e-14
precision        MN  43.630522  8.749977e-08
                 RA  48.486940  9.443735e-09
recall           MN  65.404826  3.566126e-12
                 RA  92.139229  1.088599e-17

### Post Hoc Analysis
#### $d'$

In [27]:
post_hoc_onset_dprime = ch_sdt.post_hoc_table(
    onset_nemenyi, peyes.constants.D_PRIME_STR, [GT1, GT2], ALPHA, marginal_alpha=MARGINAL_ALPHA
)
post_hoc_onset_dprime

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.      **      **    n.s.     ***     n.s.
         RA      --    n.s.     ***     ***    n.s.     ***       **
ivvt     MN  1.0000      --      **      **    n.s.     ***     n.s.
         RA  0.9999      --     ***      **    n.s.     ***        *
idt      MN  0.0030  0.0028      --    n.s.     ***    n.s.     n.s.
         RA  0.0002  0.0010      --    n.s.     ***    n.s.     n.s.
idvt     MN  0.0061  0.0057  1.0000      --      **    n.s.     n.s.
         RA  0.0008  0.0037  1.0000      --     ***    n.s.     n.s.
engbert  MN  0.9998  0.9998  0.0004  0.0010      --     ***        *
         RA  0.9982  0.9779  0.0000  0.0000      --     ***      ***
nh       MN  0.0002  0.0002  0.9981  0.9924  0.0000      --     n.s.
         RA  0.0000  0.0000  0.9982  0.9826  0.0000      --     n.s.
remodnav MN  0.0870  0.0830  0.9733  0.9904  0.0245  0.7713       --
         RA  0.0054  0.0197  0.9935  0.9997  0.0004  0.8806       --

#### $f1$

In [28]:
post_hoc_onset_f1 = ch_sdt.post_hoc_table(
    onset_nemenyi, peyes.constants.F1_STR, [GT1, GT2], ALPHA, marginal_alpha=MARGINAL_ALPHA
)
post_hoc_onset_f1

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.       *       †    n.s.      **        *
         RA      --    n.s.      **       *    n.s.      **       **
ivvt     MN  0.9837      --    n.s.    n.s.    n.s.    n.s.     n.s.
         RA  0.9675      --    n.s.    n.s.    n.s.       *     n.s.
idt      MN  0.0245  0.2565      --    n.s.       *    n.s.     n.s.
         RA  0.0036  0.1080      --    n.s.      **    n.s.     n.s.
idvt     MN  0.0543  0.3997  1.0000      --       †    n.s.     n.s.
         RA  0.0123  0.2238  1.0000      --      **    n.s.     n.s.
engbert  MN  1.0000  0.9898  0.0313  0.0674      --      **        *
         RA  1.0000  0.9297  0.0017  0.0063      --     ***       **
nh       MN  0.0048  0.0932  0.9997  0.9957  0.0065      --     n.s.
         RA  0.0010  0.0476  1.0000  0.9979  0.0004      --     n.s.
remodnav MN  0.0359  0.3193  1.0000  1.0000  0.0453  0.9987       --
         RA  0.0025  0.0866  1.0000  0.9998  0.0012  1.0000       --

#### Criterion

In [29]:
post_hoc_onset_crit = ch_sdt.post_hoc_table(
    onset_nemenyi, peyes.constants.CRITERION_STR, [GT1, GT2], ALPHA, marginal_alpha=MARGINAL_ALPHA
)
post_hoc_onset_crit

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.       *       †    n.s.    n.s.      ***
         RA      --    n.s.      **      **    n.s.    n.s.      ***
ivvt     MN  0.9985      --      **      **    n.s.       *      ***
         RA  0.9983      --     ***     ***    n.s.       *      ***
idt      MN  0.0328  0.0040      --    n.s.      **    n.s.     n.s.
         RA  0.0017  0.0001      --    n.s.     ***    n.s.     n.s.
idvt     MN  0.0530  0.0074  1.0000      --      **    n.s.     n.s.
         RA  0.0027  0.0002  1.0000      --     ***    n.s.     n.s.
engbert  MN  0.9967  1.0000  0.0028  0.0053      --       *      ***
         RA  0.9936  1.0000  0.0000  0.0001      --       *      ***
nh       MN  0.2104  0.0482  0.9955  0.9989  0.0369      --     n.s.
         RA  0.1057  0.0178  0.9315  0.9570  0.0101      --     n.s.
remodnav MN  0.0001  0.0000  0.8936  0.8288  0.0000  0.4997       --
         RA  0.0000  0.0000  0.8800  0.8331  0.0000  0.2042       --

## Offset Detection

In [30]:
offset_statistics, offset_pvalues, offset_nemenyi, offset_Ns = ch_sdt.friedman_nemenyi(
    csdt_metrics, "offset", THRESHOLD, [GT1, GT2]
)

offset_pvalues <= ALPHA

gt,MN,RA
metric,,
criterion,True,True
d_prime,True,True
f1,True,True
false_alarm_rate,True,True
precision,False,True
recall,True,True


In [31]:
pd.concat([offset_statistics, offset_pvalues], axis=1, keys=['Q', 'p']).stack(1, future_stack=True)

Q             p
metric           gt                         
criterion        MN  66.650323  1.984403e-12
                 RA  90.913043  1.957708e-17
d_prime          MN  27.630968  1.102522e-04
                 RA  72.652174  1.167045e-13
f1               MN  32.738916  1.177103e-05
                 RA  64.237425  6.173524e-12
false_alarm_rate MN  34.614776  5.116978e-06
                 RA  49.101695  7.114555e-09
precision        MN  12.534884  5.104623e-02
                 RA  46.399274  2.465212e-08
recall           MN  52.336000  1.596082e-09
                 RA  92.267155  1.023925e-17

### Post Hoc Analysis
#### $d'$

In [32]:
post_hoc_offset_dprime = ch_sdt.post_hoc_table(
    offset_nemenyi, peyes.constants.D_PRIME_STR, [GT1, GT2], ALPHA, marginal_alpha=MARGINAL_ALPHA
)
post_hoc_offset_dprime

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.    n.s.    n.s.    n.s.    n.s.     n.s.
         RA      --    n.s.     ***     ***    n.s.    n.s.       **
ivvt     MN  1.0000      --       †       †    n.s.    n.s.        †
         RA  1.0000      --     ***     ***    n.s.    n.s.       **
idt      MN  0.1251  0.0617      --    n.s.    n.s.       †     n.s.
         RA  0.0004  0.0001      --    n.s.    n.s.      **     n.s.
idvt     MN  0.1400  0.0703  1.0000      --    n.s.    n.s.     n.s.
         RA  0.0003  0.0001  1.0000      --    n.s.      **     n.s.
engbert  MN  0.8884  0.7650  0.8473  0.8670      --    n.s.     n.s.
         RA  0.7399  0.6074  0.1446  0.1259      --    n.s.     n.s.
nh       MN  1.0000  1.0000  0.0726  0.0824  0.7959      --        †
         RA  0.9995  0.9959  0.0033  0.0026  0.9390      --        *
remodnav MN  0.1208  0.0591  1.0000  1.0000  0.8408  0.0697       --
         RA  0.0098  0.0043  0.9934  0.9901  0.5496  0.0486       --

#### $f1$

In [33]:
post_hoc_offset_f1 = ch_sdt.post_hoc_table(
    offset_nemenyi, peyes.constants.F1_STR, [GT1, GT2], ALPHA, marginal_alpha=MARGINAL_ALPHA
)
post_hoc_offset_f1

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.    n.s.    n.s.    n.s.    n.s.        †
         RA      --    n.s.      **      **    n.s.    n.s.       **
ivvt     MN  1.0000      --    n.s.    n.s.    n.s.    n.s.        *
         RA  1.0000      --      **      **    n.s.    n.s.       **
idt      MN  0.1132  0.0875      --    n.s.    n.s.    n.s.     n.s.
         RA  0.0014  0.0013      --    n.s.    n.s.       *     n.s.
idvt     MN  0.1222  0.0949  1.0000      --    n.s.    n.s.     n.s.
         RA  0.0014  0.0013  1.0000      --    n.s.       *     n.s.
engbert  MN  0.8873  0.8460  0.8184  0.8330      --    n.s.     n.s.
         RA  0.8008  0.7920  0.1940  0.1879      --    n.s.     n.s.
nh       MN  0.9999  0.9996  0.2505  0.2658  0.9753      --     n.s.
         RA  0.9900  0.9888  0.0300  0.0286  0.9952      --        *
remodnav MN  0.0648  0.0488  1.0000  1.0000  0.6873  0.1583       --
         RA  0.0021  0.0020  1.0000  1.0000  0.2538  0.0427       --

#### Criterion

In [34]:
post_hoc_offset_crit = ch_sdt.post_hoc_table(
    offset_nemenyi, peyes.constants.CRITERION_STR, [GT1, GT2], ALPHA, marginal_alpha=MARGINAL_ALPHA
)
post_hoc_offset_crit

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.       *       *    n.s.    n.s.      ***
         RA      --    n.s.     ***     ***    n.s.    n.s.      ***
ivvt     MN  0.9795      --     ***     ***    n.s.    n.s.      ***
         RA  0.9195      --     ***     ***    n.s.       †      ***
idt      MN  0.0127  0.0002      --    n.s.       †    n.s.     n.s.
         RA  0.0004  0.0000      --    n.s.      **    n.s.     n.s.
idvt     MN  0.0180  0.0004  1.0000      --    n.s.    n.s.     n.s.
         RA  0.0006  0.0000  1.0000      --      **    n.s.     n.s.
engbert  MN  0.9995  0.8561  0.0596  0.0786      --    n.s.       **
         RA  1.0000  0.8259  0.0013  0.0019      --    n.s.      ***
nh       MN  0.7220  0.1789  0.6156  0.6781  0.9309      --     n.s.
         RA  0.6704  0.0680  0.1963  0.2325  0.8076      --     n.s.
remodnav MN  0.0005  0.0000  0.9924  0.9856  0.0040  0.1744       --
         RA  0.0001  0.0000  0.9999  0.9998  0.0003  0.0906       --

## Figures
### (1) Onset Distributions

In [35]:
onset_distribution_figure = ch_sdt.single_threshold_figure(
    csdt_metrics.loc[(slice(None), [peyes.constants.D_PRIME_STR, peyes.constants.F1_STR], slice(None)), :],
    peyes.constants.ONSET_STR,
    THRESHOLD,
    GT1,
    gt2=GT2,
    show_other_gt=True,
    share_x=True,
    colors={k: v[1] for k, v in LABELER_PLOTTING_CONFIG.items()},
)

onset_distribution_figure.update_layout(
    title=dict(text="fixation onsets"),
    width=1000, height=500,
    paper_bgcolor='rgba(0, 0, 0, 0)',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    margin=dict(l=10, r=10, b=10, t=10, pad=0),
)
# onset_distribution_figure.layout.annotations = []   # remove subtitles

onset_distribution_figure.show()

### (2) Offset Distributions

In [36]:
offset_distribution_figure = ch_sdt.single_threshold_figure(
    csdt_metrics.loc[(slice(None), [peyes.constants.D_PRIME_STR, peyes.constants.F1_STR], slice(None)), :],
    peyes.constants.OFFSET_STR,
    THRESHOLD,
    GT1,
    gt2=GT2,
    show_other_gt=True,
    share_x=True,
    colors={k: v[1] for k, v in LABELER_PLOTTING_CONFIG.items()},
)

offset_distribution_figure.update_layout(
    title=dict(text="fixation offsets"),
    width=1000, height=500,
    paper_bgcolor='rgba(0, 0, 0, 0)',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    margin=dict(l=10, r=10, b=10, t=10, pad=0),
)
# offset_distribution_figure.layout.annotations = []   # remove subtitles

offset_distribution_figure.show()

### (3) Final Figure - save to file
Displaying only the $d'$ distributions for saccade onsets & offsets in the same figure, to avoid showing too many subplots

In [39]:
W, H = 600, 450

discriminability_figure = make_subplots(
    rows=2, cols=1, shared_xaxes=True, subplot_titles=['Saccade Onset', 'Saccade Offset'], vertical_spacing=0.1
)

# copy onset d-prime violins into new figure
for tr in onset_distribution_figure.data:
    if tr['scalegroup'] != 'd_prime':
        # ignore non d-prime violins
        continue
    new_tr = copy.deepcopy(tr)
    new_tr['width'] = 0.8   # make violins wider so there's less space between them
    discriminability_figure.add_trace(trace=new_tr, row=1, col=1)

# copy offset d-prime violins into new figure
for tr in offset_distribution_figure.data:
    if tr['scalegroup'] != 'd_prime':
        # ignore non d-prime violins
        continue
    new_tr = copy.deepcopy(tr)
    new_tr['width'] = 0.8   # make violins wider so there's less space between them
    discriminability_figure.add_trace(trace=new_tr, row=2, col=1)

discriminability_figure.update_layout(
    title=None,
    width=W, height=H,
    paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
    yaxis=dict(showgrid=False, zeroline=False, showline=False, tickfont=dict(size=14)),
    yaxis2=dict(showgrid=False, zeroline=False, showline=False, tickfont=dict(size=14)),
    xaxis2=dict(showgrid=False, tickfont=dict(size=14), tickangle=0),
    margin=dict(l=10, r=10, b=10, t=20, pad=0),
)
# discriminability_figure.layout.annotations = []   # remove subtitles


save_fig(discriminability_figure, FIG_ID, 'right', 'saccade-discriminability', IS_SUPP)
discriminability_figure